In [33]:
#Import dependencies

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np #very efficient array and linear algebra functions
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression #Scikit-learn machine learning library for Python
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

## Load Data in

In [34]:
#Load the CSV file as a Pandas DataFrame and preview the DataFrame
df = pd.read_csv("salaries-by-college-type.csv")
df.head()

,School Name,School Type,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Massachusetts Institute of Technology (MIT),Engineering,"$72,200.00","$126,000.00","$76,800.00","$99,200.00","$168,000.00","$220,000.00"
1,California Institute of Technology (CIT),Engineering,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN
2,Harvey Mudd College,Engineering,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN
3,"Polytechnic University of New York, Brooklyn",Engineering,"$62,400.00","$114,000.00","$66,800.00","$94,300.00","$143,000.00","$190,000.00"
4,Cooper Union,Engineering,"$62,200.00","$114,000.00",NaN,"$80,200.00","$142,000.00",NaN


In [35]:
#Load the CSV file as a Pandas DataFrame and preview the DataFrame
region_df = pd.read_csv("salaries-by-region.csv")
region_df.head()

,School Name,Region,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
0,Stanford University,California,"$70,400.00","$129,000.00","$68,400.00","$93,100.00","$184,000.00","$257,000.00"
1,California Institute of Technology (CIT),California,"$75,500.00","$123,000.00",NaN,"$104,000.00","$161,000.00",NaN
2,Harvey Mudd College,California,"$71,800.00","$122,000.00",NaN,"$96,000.00","$180,000.00",NaN
3,"University of California, Berkeley",California,"$59,900.00","$112,000.00","$59,500.00","$81,000.00","$149,000.00","$201,000.00"
4,Occidental College,California,"$51,900.00","$105,000.00",NaN,"$54,800.00","$157,000.00",NaN


## Cleaning Data

In [36]:
#Remove the fields from the data set that we don't want to include in our model

del df['Mid-Career Median Salary']
del df['Mid-Career 10th Percentile Salary']
del df['Mid-Career 25th Percentile Salary']
del df['Mid-Career 75th Percentile Salary']
del df['Mid-Career 90th Percentile Salary']

In [37]:
df.head()

,School Name,School Type,Starting Median Salary
0,Massachusetts Institute of Technology (MIT),Engineering,"$72,200.00"
1,California Institute of Technology (CIT),Engineering,"$75,500.00"
2,Harvey Mudd College,Engineering,"$71,800.00"
3,"Polytechnic University of New York, Brooklyn",Engineering,"$62,400.00"
4,Cooper Union,Engineering,"$62,200.00"


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   School Name             269 non-null    object
 1   School Type             269 non-null    object
 2   Starting Median Salary  269 non-null    object
dtypes: object(3)
memory usage: 6.4+ KB


In [39]:
df['Starting Median Salary']=(df['Starting Median Salary'].replace( '[\$,)]','', regex=True )
                   .replace( '[(]','-',   regex=True ).astype(float))

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269 entries, 0 to 268
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   School Name             269 non-null    object 
 1   School Type             269 non-null    object 
 2   Starting Median Salary  269 non-null    float64
dtypes: float64(1), object(2)
memory usage: 6.4+ KB


In [41]:
#Remove the fields from the data set that we don't want to include in our model

del region_df['Mid-Career Median Salary']
del region_df['Mid-Career 10th Percentile Salary']
del region_df['Mid-Career 25th Percentile Salary']
del region_df['Mid-Career 75th Percentile Salary']
del region_df['Mid-Career 90th Percentile Salary']

In [42]:
region_df.head()

,School Name,Region,Starting Median Salary
0,Stanford University,California,"$70,400.00"
1,California Institute of Technology (CIT),California,"$75,500.00"
2,Harvey Mudd College,California,"$71,800.00"
3,"University of California, Berkeley",California,"$59,900.00"
4,Occidental College,California,"$51,900.00"


In [43]:
region_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   School Name             320 non-null    object
 1   Region                  320 non-null    object
 2   Starting Median Salary  320 non-null    object
dtypes: object(3)
memory usage: 7.6+ KB


In [44]:
region_df['Starting Median Salary']=(region_df['Starting Median Salary'].replace( '[\$,)]','', regex=True )
                   .replace( '[(]','-',   regex=True ).astype(float))

In [45]:
region_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   School Name             320 non-null    object 
 1   Region                  320 non-null    object 
 2   Starting Median Salary  320 non-null    float64
dtypes: float64(1), object(2)
memory usage: 7.6+ KB


## Join tables

In [46]:
joined_df = pd.merge(df, region_df, on=["School Name", "School Name"])
joined_df

,School Name,School Type,Starting Median Salary_x,Region,Starting Median Salary_y
0,Massachusetts Institute of Technology (MIT),Engineering,72200.0,Northeastern,72200.0
1,California Institute of Technology (CIT),Engineering,75500.0,California,75500.0
2,Harvey Mudd College,Engineering,71800.0,California,71800.0
3,"Polytechnic University of New York, Brooklyn",Engineering,62400.0,Northeastern,62400.0
4,Cooper Union,Engineering,62200.0,Northeastern,62200.0
...,...,...,...,...,...
263,Austin Peay State University,State,37700.0,Southern,37700.0
264,Pittsburg State University,State,40400.0,Midwestern,40400.0
265,Southern Utah University,State,41900.0,Western,41900.0
266,Montana State University - Billings,State,37900.0,Western,37900.0


In [58]:
joined_copy_df=joined_df.copy()
joined_copy_df

,School Name,School Type,Starting Median Salary_x,Region,Starting Median Salary_y
0,Massachusetts Institute of Technology (MIT),Engineering,72200.0,Northeastern,72200.0
1,California Institute of Technology (CIT),Engineering,75500.0,California,75500.0
2,Harvey Mudd College,Engineering,71800.0,California,71800.0
3,"Polytechnic University of New York, Brooklyn",Engineering,62400.0,Northeastern,62400.0
4,Cooper Union,Engineering,62200.0,Northeastern,62200.0
...,...,...,...,...,...
263,Austin Peay State University,State,37700.0,Southern,37700.0
264,Pittsburg State University,State,40400.0,Midwestern,40400.0
265,Southern Utah University,State,41900.0,Western,41900.0
266,Montana State University - Billings,State,37900.0,Western,37900.0


In [59]:
#delete strings
del joined_copy_df['School Name']
del joined_copy_df['School Type']
del joined_copy_df['Region']

In [60]:
joined_copy_df.head()

,Starting Median Salary_x,Starting Median Salary_y
0,72200.0,72200.0
1,75500.0,75500.0
2,71800.0,71800.0
3,62400.0,62400.0
4,62200.0,62200.0


## creating clusters to determin how many starting salary buckets to have

In [61]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas

In [62]:
# standardized data with StandardScaler
joined_copy_scaled = StandardScaler().fit_transform(joined_copy_df)
print(joined_copy_scaled[0:5])

[[4.08708144 4.08708144]
 [4.60272572 4.60272572]
 [4.0245791  4.0245791 ]
 [2.55577419 2.55577419]
 [2.52452302 2.52452302]]


In [63]:
# Initialize PCA model
pca = PCA(n_components=2)

In [64]:
# Get two principal components for the data.
joined_pca = pca.fit_transform(joined_copy_scaled)

In [66]:
df_joined_pca = pd.DataFrame(
    data=joined_pca, columns=["principal component 1", "principal component 2"]
)
df_joined_pca.head()

,principal component 1,principal component 2
0,5.780006,2.419311e-16
1,6.509237,-1.028429e-16
2,5.691614,1.213146e-17
3,3.614411,-1.032345e-16
4,3.570215,-2.606256e-17


In [67]:
# Fetch the explained variance
pca.explained_variance_ratio_

array([1.00000000e+00, 4.70591503e-34])

In [68]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_joined_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [72]:
# Initialize the K-means model
model = KMeans(n_clusters=3, random_state=0)

# Fit the model
model.fit(df_joined_pca)

# Predict clusters
predictions = model.predict(df_joined_pca)

# Add the predicted class columns
df_joined_pca["class"] = model.labels_
df_joined_pca.head()

,principal component 1,principal component 2,class
0,5.780006,2.419311e-16,2
1,6.509237,-1.028429e-16,2
2,5.691614,1.213146e-17,2
3,3.614411,-1.032345e-16,2
4,3.570215,-2.606256e-17,2


In [73]:
df_joined_pca.hvplot.scatter(
    x="principal component 1",
    y="principal component 2",
    hover_cols=["class"],
    by="class",
)
# graph shows we can categorized starting salary into three clusters. these labels can be the 
# classification our model predicts.

:NdOverlay   [class]
   :Scatter   [principal component 1]   (principal component 2,class)

## what was here before

In [14]:
#Replace the categorical data with one-hot encoded data
features_df = pd.get_dummies(df, columns=['School Type'])
features_df.head()

,Starting Median Salary,School Type_Engineering,School Type_Ivy League,School Type_Liberal Arts,School Type_Party,School Type_State
0,72200.0,1,0,0,0,0
1,75500.0,1,0,0,0,0
2,71800.0,1,0,0,0,0
3,62400.0,1,0,0,0,0
4,62200.0,1,0,0,0,0


In [11]:
#Create the X and y arrays
X = features_df.to_numpy()
y = df['School Type'].to_numpy()

In [12]:
#Split into a training set and a test set
from sklearn.model_selection import train_test_split

#training set will be the larger portion of the data, typically 70% or more
#after the split will have four sets of data: x_train, x_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
model = ensemble.GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.5,
    max_depth=6,
    min_samples_leaf=9,
    max_features=0.1,
    loss='huber'
)

model.fit(X_train,y_train)

ValueError: could not convert string to float: 'State'